In [ ]:
import sys
sys.path.append('../src')
from vsa import VSA
import numpy as np

vsa = VSA(dim=2048)

# Reuse HKM from ICT notebook (recreate for independence)
np.random.seed(42)
concepts = [f'concept_{i}' for i in range(100)]
atomic_vectors = {concept: np.random.normal(0, 1/np.sqrt(vsa.dim), vsa.dim) for concept in concepts}

facts = []
for i in range(0, 100, 2):
    role = atomic_vectors[concepts[i]]
    filler = atomic_vectors[concepts[i+1]]
    bound = vsa.bind(role, filler)
    facts.append(bound)

M = vsa.bundle(facts)

In [ ]:
def generate_projections(vector, k, sub_dimension):
    projections = []
    for _ in range(k):
        proj_matrix = np.random.normal(0, 1/np.sqrt(sub_dimension), (sub_dimension, vsa.dim))
        projected = np.dot(proj_matrix, vector)
        projections.append(projected)
    return projections

def get_consensus(role, expected_filler, projections, threshold=0.5):
    agreements = 0
    for proj in projections:
        retrieved = vsa.unbind(role, proj)  # Note: unbind needs to handle projected dims, assume vsa.unbind updated if needed
        coherence = np.dot(retrieved, expected_filler) / (np.linalg.norm(retrieved) * np.linalg.norm(expected_filler))
        if coherence > threshold:
            agreements += 1
    return (agreements / len(projections)) > 0.8

In [ ]:
import matplotlib.pyplot as plt

# Create a spurious fact (role and filler not in M)
spurious_role = np.random.normal(0, 1/np.sqrt(vsa.dim), vsa.dim)
spurious_filler = np.random.normal(0, 1/np.sqrt(vsa.dim), vsa.dim)

# Parameters
sub_dimension = 512  # Lower dimension for projections
num_trials = 100  # Number of trials per k to estimate error rate
ks = range(1, 26)  # Vary k from 1 to 25
error_rates = []

for k in ks:
    errors = 0
    for _ in range(num_trials):
        projections = generate_projections(M, k, sub_dimension)
        if get_consensus(spurious_role, spurious_filler, projections):
            errors += 1
    error_rate = errors / num_trials
    error_rates.append(error_rate)

plt.plot(ks, error_rates, marker='o')
plt.xlabel('Number of Projections (k)')
plt.ylabel('Error Rate')
plt.title('Error Rate vs. Number of Projections')
plt.show()